In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama.chat_models import ChatOllama
from langchain_core.tools import tool


In [2]:

model = 'llama3.2:3b-instruct-fp16'

llm = ChatOllama(
    model=model,
    temperature=0,
)


In [3]:
prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should use a tools if its necessary "
        "if you can answer the question without using a tool, answer it "
        "use the history and contexxt for relevant answers"
        "user's CURRENT question, not previous questions."
        "{context1} and {context2} "
        
    )),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [4]:
pipeline=(prompt | llm)


In [5]:
@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x



tools = [add, subtract, multiply, exponentiate]

name2tool = {tool.name: tool.func for tool in tools}

In [6]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

class BufferWindowMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)

    def __init__(self, k: int):
        super().__init__(k=k)
        print(f"Initializing BufferWindowMessageHistory with k={k}")

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history, removing any messages beyond
        the last `k` messages.
        """
        self.messages.extend(messages)
        self.messages = self.messages[-self.k:]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

In [7]:
chat_map = {}
def get_chat_history(session_id: str, k: int = 4) -> BufferWindowMessageHistory:
    if session_id not in chat_map:
        chat_map[session_id] = BufferWindowMessageHistory(k=k)
    return chat_map[session_id]

In [8]:
from langchain.agents import create_tool_calling_agent

tools = [add, subtract, multiply,exponentiate]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

In [17]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False
)

In [18]:
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables import RunnableWithMessageHistory



In [19]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
embeddings = OllamaEmbeddings(model="llama3.2:3b-instruct-fp16")

vector1= DocArrayInMemorySearch.from_texts(
    ["hello how are you", "my name is haziq"],
    embedding=embeddings
)
vector2= DocArrayInMemorySearch.from_texts(
    ["hello how are you", "my nick name is saleem"],embedding=embeddings)


In [20]:
from langchain_core.runnables import RunnablePassthrough,RunnableParallel
retriever1 = vector1.as_retriever(search_kwargs={"k": 2})
retriever2 = vector2.as_retriever(search_kwargs={"k": 2})
retriever_parallel = RunnableParallel({
    "context1":retriever1, 'context2':retriever2, 'input':RunnablePassthrough()
}
)


In [21]:
from langchain_core.runnables import RunnableLambda
from langchain.schema.output_parser import StrOutputParser
from operator import itemgetter
output_parser = StrOutputParser()

def merge_context(inputs: dict) -> dict:
    return {
        "input": inputs["input"],
        "context1": "\n".join([doc.page_content for doc in inputs["context1"]]),
        "context2": "\n".join([doc.page_content for doc in inputs["context2"]]),
        "history": inputs.get("history", []),
    }

merge_context_runnable = RunnableLambda(merge_context)
rag_pipeline = retriever_parallel | merge_context_runnable | agent_executor | itemgetter('output') |output_parser



In [22]:
pipeline_with_history = RunnableWithMessageHistory(
    rag_pipeline,
    get_session_history=get_chat_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="The number of messages to keep in the history",
            default=4,
        )
    ]
)

In [23]:
pipeline_with_history.invoke(
    {"input": "what is my nickname ",
    }, 
    config={"configurable": {"session_id": "usr", "k": 4}}
)

ValidationError: 1 validation error for exponentiate
x
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='Saleem', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_parsing